**Id:** 55031 **Nome:** RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI<br>
**Id:** 52662 **Nome:** CÁLCULO DIFERENCIAL E INTEGRAL I<br>
**Id:** 55022 **Nome:** FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO I<br>

In [1]:
import numpy as np
import pandas as pd

In [7]:
#Função que insere 'vazio' no campo null. Sem isso os dataframes iram quebrar.
def TratarDfVaziosProf(df):
    if(df.empty):
        df = pd.DataFrame()
        df = df.append({'id_servidor':0, 'nome':'vazio', 'formacao':'vazio', 'tipo_jornada_trabalho':'vazio', 'vinculo':'vazio', 'classe_funcional':'vazio', 'lotacao': 'vazio'}, ignore_index=True)
        return df
    return df

In [8]:
pd_disc = pd.read_csv('disciplinas.csv')
#pd_disc.iloc[0]
pd_disc

,id_disc,cod_disc,nome_disc,hora_disc
0,55031,IMD0019,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,180
1,52662,IMD0024,CÁLCULO DIFERENCIAL E INTEGRAL I,90
2,55022,IMD0028,FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO I,90


In [84]:
pd_turmas2017_1 = pd.read_csv('turmas-2017.1.csv', error_bad_lines = False, warn_bad_lines = False, encoding = "utf-8", sep=";", usecols=['id_turma', 'codigo_turma', 'id_docente_interno', 'id_componente_curricular', 'local', 'ano', 'periodo', 'descricao_horario', 'total_solicitacoes', 'capacidade_aluno', 'situacao_turma'])
pd_turmas2017_1.head(3)

,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,ano,periodo,descricao_horario,total_solicitacoes,capacidade_aluno,situacao_turma
0,57593168,1,26652.0,61150,Sala 24,2017,1,2T12,NaN,50.0,CONSOLIDADA
1,57593169,2,21707.0,61150,Sala 24,2017,1,4T12,NaN,50.0,CONSOLIDADA
2,57593170,1,21707.0,61152,Sala 24,2017,1,3T56,0.0,50.0,EXCLUÍDA


In [9]:
pd_docentes = pd.read_csv('docentes.csv', encoding = "utf-8", sep=";", usecols=['id_servidor', 'nome', 'formacao', 'tipo_jornada_trabalho', 'vinculo', 'lotacao'])
pd_docentes.head(3)

,id_servidor,nome,formacao,tipo_jornada_trabalho,vinculo,lotacao
0,21041,GILDETE AZEVEDO,DESCONHECIDA,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA
1,5753261,MARIA NUBIA PESSOA,ESPECIALIZAÇÃO,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA
2,5752617,RUTILENE SANTOS DE SOUSA,MESTRADO,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA


In [14]:
all_turma_matriculados = pd.read_csv('all_turma_matriculados.csv', encoding = "utf-8", sep=",")

framesAux = []
for index_turma, row_turma in all_turma_matriculados.iterrows():
    pd_turmas = pd.read_csv(row_turma['csv_turma'], error_bad_lines = False, warn_bad_lines = False, encoding = "utf-8", sep=";", usecols=['id_turma', 'codigo_turma', 'id_docente_interno', 'id_componente_curricular', 'local', 'ano', 'periodo', 'descricao_horario', 'total_solicitacoes', 'capacidade_aluno', 'situacao_turma'])
    framesInter = []
    for index_pd_disc, row_pd_disc in pd_disc.iterrows():

        id_disc = row_pd_disc['id_disc']
        list_turmas = pd_turmas[pd_turmas['id_componente_curricular'] == id_disc] # filtra todas as turmas com o id da disciplina
        list_turmas_disc = list_turmas.assign(nome_disc=row_pd_disc['nome_disc']) # add uma coluna com o nome da disciplina

        listProf = []
        for index, row in list_turmas_disc.iterrows():
            
            prof = pd_docentes[pd_docentes['id_servidor'] == int(row["id_docente_interno"])]
            listProf.append(TratarDfVaziosProf(prof))

        df_profs = pd.concat(listProf)
        list_turmas_disc_prof = list_turmas_disc.assign(
            nome_prof = df_profs['nome'].tolist(),
            id_prof = df_profs['id_servidor'].tolist(),
            formacao_prof = df_profs['formacao'].tolist(),
            tipo_jornada_trabalho_prof = df_profs['tipo_jornada_trabalho'].tolist(),
            vinculo_prof = df_profs['vinculo'].tolist(),
            lotacao_prof = df_profs['lotacao'].tolist())
        framesInter.append(list_turmas_disc_prof)
    
    df_turma_professorAux = pd.concat(framesInter) # concatena todos os dataframes
    framesAux.append(df_turma_professorAux)
    
df_turma_professor = pd.concat(framesAux) 
df_turma_professor.to_csv('df_turma_professor.csv', encoding='UTF-8')

In [87]:
url = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171.csv'
pd_alunos_2017_1 = pd.read_csv(url, encoding = "utf-8", sep=";")
pd_alunos_2017_1.head(3)

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
0,57600868,a4a014ab217b40735e3caef890a56eee,4970.0,1.0,0.0,f,0.0,0.0,0.0,REPROVADO
1,57600868,a4a014ab217b40735e3caef890a56eee,4970.0,2.0,0.0,f,0.0,0.0,0.0,REPROVADO
2,57600868,89dba505ee16aab34cd076ab247459fa,4970.0,1.0,0.0,f,0.0,0.0,0.0,REPROVADO


In [88]:
frames = []
for item in df_turma_professor_2017_1['id_turma'].unique():
    df_alunos = pd_alunos_2017_1[pd_alunos_2017_1['id_turma'] == item]
    frames.append(df_alunos)
df_alunos = pd.concat(frames) # concatena todos os dataframes
df_alunos.to_csv('alunos_2017_1.csv', encoding='UTF-8')
df_alunos.head(5)

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
136535,57597051,585a86076d71152014a6dd7fee234bd0,92127264.0,3.0,2.5,t,0.0,5.0,10.0,APROVADO POR NOTA
136536,57597051,585a86076d71152014a6dd7fee234bd0,92127264.0,2.0,5.0,f,0.0,5.0,10.0,APROVADO POR NOTA
136537,57597051,585a86076d71152014a6dd7fee234bd0,92127264.0,1.0,5.5,f,0.0,5.0,10.0,APROVADO POR NOTA
136538,57597051,4aa63133c963bd0936a731846d0be1b6,92127264.0,3.0,7.3,f,0.0,7.3,8.0,APROVADO
136539,57597051,4aa63133c963bd0936a731846d0be1b6,92127264.0,2.0,9.3,f,0.0,7.3,8.0,APROVADO


In [89]:
listMean = []
listMin = []
list1q = []
list2q = []
list3q = []
listmax = []

for turma in df_turma_professor_2017_1['id_turma']:
    df_turma_alunos_3notas = df_alunos[df_alunos['id_turma'] == turma] #nesse df tá com as 3 notas do aluno
    
    if(df_turma_alunos_3notas['discente'].size > 0):
        
        df_turma_alunos_final = df_turma_alunos_3notas.drop_duplicates(subset=['discente']) #retiro 2 linhas do mesmo aluno.
        media_final_summary = df_turma_alunos_final.describe()['media_final']
        
        listMean.append(media_final_summary['mean'])
        listMin.append(media_final_summary['min'])
        list1q.append(media_final_summary['25%'])
        list2q.append(media_final_summary['50%'])
        list3q.append(media_final_summary['75%'])
        listmax.append(media_final_summary['max'])

    else: #ocorre quando não encontra uma turma
        listMean.append(0)
        listMin.append(0)
        list1q.append(0)
        list2q.append(0)
        list3q.append(0)
        listmax.append(0)


df_turma_professor_2017_1 = df_turma_professor_2017_1.assign(
    disc_mean = listMean,
    disc_min = listMin,
    disc_q1 = list1q,
    disc_q2 = list2q,
    disc_q3 = list3q,
    disc_max = listmax
    )
df_turma_professor_2017_1.to_csv('turma_professor_2017_1.csv', encoding='UTF-8')
df_turma_professor_2017_1

,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,ano,periodo,descricao_horario,total_solicitacoes,capacidade_aluno,...,lotacao_prof,nome_prof,tipo_jornada_trabalho_prof,vinculo_prof,disc_max,disc_mean,disc_min,disc_q1,disc_q2,disc_q3
4880,57597051,1,5757585.0,55031,B203 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),22.0,83.0,...,vazio,vazio,vazio,vazio,8.9,4.070968,0.0,2.425,4.90,5.500
4881,57597051,1,5756816.0,55031,B203 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),22.0,83.0,...,INSTITUTO METROPOLE DIGITAL,ALYSON MATHEUS DE CARVALHO SOUZA,Dedicação exclusiva,Ativo Permanente,8.9,4.070968,0.0,2.425,4.90,5.500
4888,57597055,2,5756816.0,55031,B204 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),16.0,81.0,...,INSTITUTO METROPOLE DIGITAL,ALYSON MATHEUS DE CARVALHO SOUZA,Dedicação exclusiva,Ativo Permanente,9.2,5.417460,0.0,5.000,5.80,7.050
4889,57597055,2,5759154.0,55031,B204 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),16.0,81.0,...,vazio,vazio,vazio,vazio,9.2,5.417460,0.0,5.000,5.80,7.050
4891,57597057,3,5756386.0,55031,B204,2017,1,246T1234 (13/02/2017 - 01/07/2017),50.0,89.0,...,INSTITUTO METROPOLE DIGITAL,CHARLES ANDRYE GALVAO MADEIRA,Dedicação exclusiva,Ativo Permanente,8.1,2.679310,0.0,0.750,1.75,5.075
4892,57597057,3,5757585.0,55031,B204,2017,1,246T1234 (13/02/2017 - 01/07/2017),50.0,89.0,...,vazio,vazio,vazio,vazio,8.1,2.679310,0.0,0.750,1.75,5.075
4893,57597058,4,5758787.0,55031,B203 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),25.0,87.0,...,INSTITUTO METROPOLE DIGITAL,AUGUSTO JORGE DE MACEDO,20 horas semanais,Ativo Permanente,8.1,5.620000,0.0,5.300,6.00,7.050
4894,57597058,4,5757583.0,55031,B203 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),25.0,87.0,...,vazio,vazio,vazio,vazio,8.1,5.620000,0.0,5.300,6.00,7.050
4898,57597062,5,5757583.0,55031,B204 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),30.0,77.0,...,vazio,vazio,vazio,vazio,8.7,3.444615,0.0,0.900,3.60,5.700
4899,57597062,5,5757669.0,55031,B204 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),30.0,77.0,...,vazio,vazio,vazio,vazio,8.7,3.444615,0.0,0.900,3.60,5.700
